### Timoshenko's model of beams

Extension of the Euler-Bernoulli's model of beams, accounting shear forces

In [1]:
# Importing SymPy
import sympy as smp

In [2]:
# Defining symbols
x, y, z, dx, dy, dz = smp.symbols("x y z dx dy dz", real = True)
r = smp.Matrix([x, y, z]) # Position

vy = smp.Function("v_y", real = True)(z)
vz = smp.Function("v_z", real = True)(z)
v = smp.Matrix([0, vy, vz]) # Translation

phi = smp.Function("phi", real = True)(z)
psi = smp.Function("psi", real = True)(z)

# Rotation matrix around the x-axis (Rx)
Rx = smp.Matrix([(1, 0, 0),
                 (0, 1, - phi),
                 (0, phi, 1)])

In [3]:
# General displacement: translation + rotation around the x-axis
u = v + ((Rx - smp.eye(3)) * (r + v))
u

Matrix([
[                            0],
[-(z + v_z(z))*phi(z) + v_y(z)],
[ (y + v_y(z))*phi(z) + v_z(z)]])

In [4]:
# General displacement on the z-axis
u_axis = u.subs([(x, 0),
                 (y, 0)])
u_axis

Matrix([
[                            0],
[-(z + v_z(z))*phi(z) + v_y(z)],
[       phi(z)*v_y(z) + v_z(z)]])

In [5]:
# Difference displacement
diff = (u - u_axis).applyfunc(smp.simplify)
diff

Matrix([
[       0],
[       0],
[y*phi(z)]])

In [6]:
# Defining the field a
ay = smp.Function("a_y", real = True)(z)
az = smp.Function("a_z", real = True)(z)
a = smp.Matrix([0, ay, az])

In [7]:
# Defining u in terms of a components
u = a + diff
u

Matrix([
[                0],
[           a_y(z)],
[y*phi(z) + a_z(z)]])

In [8]:
# Computing the strain tensor field using its definition
e11 = smp.diff(u[0], x)
e22 = smp.diff(u[1], y)
e33 = smp.diff(u[2], z)
e12 = (smp.diff(u[1], x) + smp.diff(u[0], y)) / 2
e13 = (smp.diff(u[0], z) + smp.diff(u[2], x)) / 2
e23 = (smp.diff(u[1], z) + smp.diff(u[2], y)) / 2

e = smp.Matrix([(e11, e12, e13),
                (e12, e22, e23),
                (e13, e23, e33)])
e

Matrix([
[0,                                  0,                                               0],
[0,                                  0,              phi(z)/2 + Derivative(a_y(z), z)/2],
[0, phi(z)/2 + Derivative(a_y(z), z)/2, y*Derivative(phi(z), z) + Derivative(a_z(z), z)]])

In [9]:
# Computing the stress tensor field
E = smp.symbols("E", real = True, constant = True) # Young coefficient

sigma = E * e # Poisson coefficient nu = 0
sigma

Matrix([
[0,                                      0,                                                   0],
[0,                                      0,              E*(phi(z)/2 + Derivative(a_y(z), z)/2)],
[0, E*(phi(z)/2 + Derivative(a_y(z), z)/2), E*(y*Derivative(phi(z), z) + Derivative(a_z(z), z))]])

In [10]:
# Defining I and A
I, A = smp.symbols('I A', real = True)

In [11]:
# Computing dN
dN = sigma[2, 2]
dN

E*(y*Derivative(phi(z), z) + Derivative(a_z(z), z))

In [ ]:
# Computing N (Macroscopic internal force along z-axis)
N = A * E * smp.diff(az, z, 1)
N

A*E*Derivative(a_z(z), z)

In [13]:
# Computing dT
dT = sigma[1, 2]
dT

E*(phi(z)/2 + Derivative(a_y(z), z)/2)

In [ ]:
# Computing T (Macroscopic internal force along y-axis)
T = A * dT
T

A*E*(phi(z)/2 + Derivative(a_y(z), z)/2)

In [15]:
# Computing dM
dM = (r.cross(smp.Matrix([sigma[0, 2], sigma[1, 2], sigma[2, 2]])).subs(x, 0)[0].expand()).collect(z)
dM

E*y**2*Derivative(phi(z), z) + E*y*Derivative(a_z(z), z) + z*(-E*phi(z)/2 - E*Derivative(a_y(z), z)/2)

In [ ]:
# Computing M (Macroscopic torsional moment x-component)
M = E * (I * smp.diff(phi, z, 1) - z * A / 2 * (phi + smp.diff(ay, z, 1)))
M

E*(-A*z*(phi(z) + Derivative(a_y(z), z))/2 + I*Derivative(phi(z), z))

In [17]:
# Defining loads
t = smp.Function("t", real = True)(z)
p = smp.Function("p", real = True)(z)

In [18]:
# First differential equation (d/dz N = - t)
smp.Eq(smp.diff(N, z, 1), - t)

Eq(A*E*Derivative(a_z(z), (z, 2)), -t(z))

In [19]:
# Second differential equation (d/dz T = - p)
smp.Eq(smp.diff(T, z, 1), - p)

Eq(A*E*(Derivative(a_y(z), (z, 2))/2 + Derivative(phi(z), z)/2), -p(z))

In [20]:
# Third differential equation (d/dz M = 0)
smp.Eq(smp.diff(M, z, 1).collect(z).subs(A * z, 0), 0)

Eq(E*(-A*(phi(z) + Derivative(a_y(z), z))/2 + I*Derivative(phi(z), (z, 2))), 0)

### Maximo Code below

 ```maxima

 -->	/* Cantilever beam with a concentrated force at the free end */
	/* Differential equations */
	eq1: E * I * 'diff(φ(z), z, 2) - E / 2 * A * ('diff(a(z), z, 1) + φ(z)) = 0;
	eq2: E / 2 * A * ('diff(φ(z), z, 1) + 'diff(a(z), z, 2)) = 0;
(eq1)	E·I·('diff(φ(z),z,2))-(A·E·('diff(a(z),z,1)+φ(z)))/2=0

(eq2)	(A·E·('diff(φ(z),z,1)+'diff(a(z),z,2)))/2=0
 -->	/* Solving the system */
	sol: desolve([eq1, eq2], [φ(z), a(z)]);
(sol)	[φ(z)=z·(at('diff(φ(z),z,1),z=0))+(A·z^2·(at('diff(a(z),z,1),z=0)))/(4·I)+(φ(0)·A·z^2)/(4·I)+φ(0),a(z)=-((z^2·(at('diff(φ(z),z,1),z=0)))/2)-(A·z^3·(at('diff(a(z),z,1),z=0)))/(12·I)+z·(at('diff(a(z),z,1),z=0))-(φ(0)·A·z^3)/(12·I)+a(0)]
 -->	phi_sol: rhs(sol[1])$;
 -->	a_sol: rhs(sol[2])$;
 -->	/* Applying first two boundary conditions */
	a_sol: subst([a(0) = 0, φ(0) = 0, 'diff(a(z), z, 1) = C, 'diff(φ(z), z, 1) = K], a_sol);
	phi_sol: subst([a(0) = 0, φ(0) = 0, 'diff(a(z), z, 1) = C, 'diff(φ(z), z, 1) = K], phi_sol);
(a_sol)	-((A·C·z^3)/(12·I))-(K·z^2)/2+C·z

(phi_sol)	(A·C·z^2)/(4·I)+K·z
 -->	/* Third boundary condition */
	bc1: ev(diff(phi_sol, z, 1), z = L) = 0;
(bc1)	(A·C·L)/(2·I)+K=0
 -->	/* Fourth boundary condition */
	bc2: A * E / 2 * (ev(phi_sol, z = L) + ev(diff(a_sol, z, 1), z = L)) = F;
(bc2)	(A·C·E)/2=F
 -->	/* Solving for C and K constants */
	solve([bc1, bc2], [C, K])[1];
(%o10)	[C=(2·F)/(A·E),K=-((F·L)/(E·I))]
 -->	C_sol: rhs(solve([bc1, bc2], [C, K])[1][1])$
 -->	K_sol: rhs(solve([bc1, bc2], [C, K])[1][2])$
 -->	/* Final a(z) */
	a_sol: factor(subst([C = C_sol, K = K_sol], a_sol));
(a_sol)	-((F·z·(A·z^2-3·A·L·z-12·I))/(6·A·E·I))
 -->	/* Final φ(z) */
	phi_sol: factor(subst([C = C_sol, K = K_sol], phi_sol));
(phi_sol)	(F·z·(z-2·L))/(2·E·I)
 -->	/******************************************************/
	/* Cantilever beam with a distributed load */
	/* Differential equations */
	eq1: E * I * 'diff(φ(z), z, 2) - E / 2 * A * ('diff(a(z), z, 1) + φ(z)) = 0;
	eq2: E / 2 * A * ('diff(φ(z), z, 1) + 'diff(a(z), z, 2)) = - p;
	
	/* Solving the system */
	sol: desolve([eq1, eq2], [φ(z), a(z)]);
	
	phi_sol: rhs(sol[1])$;
	a_sol: rhs(sol[2])$;
	
	
(eq1)	E·I·('diff(φ(z),z,2))-(A·E·('diff(a(z),z,1)+φ(z)))/2=0

(eq2)	(A·E·('diff(φ(z),z,1)+'diff(a(z),z,2)))/2=-p

(sol)	[φ(z)=z·(at('diff(φ(z),z,1),z=0))+(z^2·(A·(at('diff(a(z),z,1),z=0))+φ(0)·A))/(12·I)+(A·z^2·(at('diff(a(z),z,1),z=0)))/(6·I)-(p·z^3)/(6·E·I)+(φ(0)·A·z^2)/(6·I)+φ(0),a(z)=-((z^2·(2·A·E·(at('diff(φ(z),z,1),z=0))+4·p))/(24·A·E))-(z^2·(A·E·(at('diff(φ(z),z,1),z=0))+2·p))/(6·A·E)-
(z^2·(at('diff(φ(z),z,1),z=0)))/4-(z^3·(A·(at('diff(a(z),z,1),z=0))+φ(0)·A))/(48·I)-(A·z^3·(at('diff(a(z),z,1),z=0)))/(16·I)+z·(at('diff(a(z),z,1),z=0))+(p·z^4)/(24·E·I)-(φ(0)·A·z^3)/(16·I)-(p·z^2)/(2·A·E)+a(0)]
 -->	/* Applying first two boundary conditions */
	a_sol: subst([a(0) = 0, φ(0) = 0, 'diff(a(z), z, 1) = C, 'diff(φ(z), z, 1) = K], a_sol);
	phi_sol: subst([a(0) = 0, φ(0) = 0, 'diff(a(z), z, 1) = C, 'diff(φ(z), z, 1) = K], phi_sol);
(a_sol)	(p·z^4)/(24·E·I)-(A·C·z^3)/(12·I)-((4·p+2·A·E·K)·z^2)/(24·A·E)-((2·p+A·E·K)·z^2)/(6·A·E)-(p·z^2)/(2·A·E)-(K·z^2)/4+C·z

(phi_sol)	-((p·z^3)/(6·E·I))+(A·C·z^2)/(4·I)+K·z
 -->	
 -->	/* Third boundary condition */
	bc1: ev(diff(phi_sol, z, 1), z = L) = 0;
(bc1)	-((L^2·p)/(2·E·I))+(A·C·L)/(2·I)+K=0
 -->	/* Fourth boundary condition */
	bc2: A * E / 2 * (ev(phi_sol, z = L) + ev(diff(a_sol, z, 1), z = L)) = 0;
(bc2)	(A·E·(-((L·(4·p+2·A·E·K))/(12·A·E))-(L·(2·p+A·E·K))/(3·A·E)-(L·p)/(A·E)+(K·L)/2+C))/2=0
 -->	/* Solving for C and K constants */
	solve([bc1, bc2], [C, K])[1];
(%o24)	[C=(2·L·p)/(A·E),K=-((L^2·p)/(2·E·I))]
 -->	C_sol: rhs(solve([bc1, bc2], [C, K])[1][1])$
	K_sol: rhs(solve([bc1, bc2], [C, K])[1][2])$
 -->	/* Final a(z) */
	a_sol: factor(subst([C = C_sol, K = K_sol], a_sol));
(a_sol)	(p·z·(A·z^3-4·A·L·z^2+6·A·L^2·z-24·I·z+48·I·L))/(24·A·E·I)
 -->	/* Final φ(z) */
	phi_sol: factor(subst([C = C_sol, K = K_sol], phi_sol));
(phi_sol)	-((p·z·(z^2-3·L·z+3·L^2))/(6·E·I))
 -->	/* Simply supported beam with a distributed load */
	/* Differential equations */
	eq1: E * I * 'diff(φ(z), z, 2) - E / 2 * A * ('diff(a(z), z, 1) + φ(z)) = 0$;
	eq2: E / 2 * A * ('diff(φ(z), z, 1) + 'diff(a(z), z, 2)) = - p$;
 -->	/* Solving the system */
	sol: desolve([eq1, eq2], [φ(z), a(z)]);
(sol)	[φ(z)=z·(at('diff(φ(z),z,1),z=0))+(z^2·(A·(at('diff(a(z),z,1),z=0))+φ(0)·A))/(12·I)+(A·z^2·(at('diff(a(z),z,1),z=0)))/(6·I)-(p·z^3)/(6·E·I)+(φ(0)·A·z^2)/(6·I)+φ(0),a(z)=-((z^2·(2·A·E·(at('diff(φ(z),z,1),z=0))+4·p))/(24·A·E))-(z^2·(A·E·(at('diff(φ(z),z,1),z=0))+2·p))/(6·A·E)-
(z^2·(at('diff(φ(z),z,1),z=0)))/4-(z^3·(A·(at('diff(a(z),z,1),z=0))+φ(0)·A))/(48·I)-(A·z^3·(at('diff(a(z),z,1),z=0)))/(16·I)+z·(at('diff(a(z),z,1),z=0))+(p·z^4)/(24·E·I)-(φ(0)·A·z^3)/(16·I)-(p·z^2)/(2·A·E)+a(0)]
 -->	phi_sol: rhs(sol[1])$;
	a_sol: rhs(sol[2])$;
 -->	/* Applying the first boundary condition */
	a_sol: subst([a(0) = 0, φ(0) = J, 'diff(a(z), z, 1) = C, 'diff(φ(z), z, 1) = K], a_sol);
	phi_sol: subst([a(0) = 0, φ(0) = J, 'diff(a(z), z, 1) = C, 'diff(φ(z), z, 1) = K], phi_sol);
(a_sol)	(p·z^4)/(24·E·I)-((A·J+A·C)·z^3)/(48·I)-(A·J·z^3)/(16·I)-(A·C·z^3)/(16·I)-((4·p+2·A·E·K)·z^2)/(24·A·E)-((2·p+A·E·K)·z^2)/(6·A·E)-(p·z^2)/(2·A·E)-(K·z^2)/4+C·z

(phi_sol)	-((p·z^3)/(6·E·I))+((A·J+A·C)·z^2)/(12·I)+(A·J·z^2)/(6·I)+(A·C·z^2)/(6·I)+K·z+J
 -->	/* Applying the second boundary condition */
	bc1: subst(z = 0, diff(phi_sol, z, 1)) = 0;
(bc1)	K=0
 -->	/* Applying the third boundary condition */
	bc2: ev(a_sol, z = L) = 0;
(bc2)	-((L^2·(4·p+2·A·E·K))/(24·A·E))-(L^2·(2·p+A·E·K))/(6·A·E)+(L^4·p)/(24·E·I)-(L^2·p)/(2·A·E)-((A·J+A·C)·L^3)/(48·I)-(A·J·L^3)/(16·I)-(A·C·L^3)/(16·I)-(K·L^2)/4+C·L=0
 -->	/* Applying the fourth boundary condition */
	bc3: ev(diff(phi_sol, z, 1), z = L) = 0;
(bc3)	-((L^2·p)/(2·E·I))+((A·J+A·C)·L)/(6·I)+(A·J·L)/(3·I)+(A·C·L)/(3·I)+K=0
 -->	/* Solving for J, C and K constants */
	solve([bc1, bc2, bc3], [J, C, K])[1];
(%o39)	[J=-((L^3·p)/(24·E·I)),C=((A·L^3+24·I·L)·p)/(24·A·E·I),K=0]
 -->	J_sol: rhs(solve([bc1, bc2, bc3], [J, C, K])[1][1])$
	C_sol: rhs(solve([bc1, bc2, bc3], [J, C, K])[1][2])$
	K_sol: rhs(solve([bc1, bc2, bc3], [J, C, K])[1][3])$
 -->	/* Final a(z) */
	a_sol: factor(subst([J = J_sol, C = C_sol, K = K_sol], a_sol));
(a_sol)	(p·z·(z-L)·(A·z^2-A·L·z-A·L^2-24·I))/(24·A·E·I)
 -->	/* Final φ(z) */
	phi_sol: factor(subst([J = J_sol, C = C_sol, K = K_sol], phi_sol));
(phi_sol)	-((p·(2·z-L)·(2·z^2-2·L·z-L^2))/(24·E·I))

```